# How to view and update past graph state
- https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/time-travel/

### Init Agent

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph import MessagesState, START
from langgraph.prebuilt import ToolNode
from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

@tool
def play_song_on_spotify(song: str):
    """Play a song on Spotify"""
    return f"Successfully played {song} on Spotify"

@tool
def play_song_on_apple(song: str):
    """Play a song on Apple Music"""
    return f"Successfully played {song} on Apple Music"

tools = [play_song_on_spotify, play_song_on_apple]
tool_node = ToolNode(tools)

# Setup the model
model = ChatOpenAI(model="gpt-4o-mini")
model = model.bind_tools(tools, parallel_tool_calls=False)

# Define nodes and conditional edges

# DEfine the function the determines whether to continue or not
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]

    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    else:
        return "continue"

# Define the function that calls the model
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)

    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, {"continue": "action", "end": END})
workflow.add_edge("action", "agent")

memory = MemorySaver()

app = workflow.compile(checkpointer=memory)

### Interacting with Agent

In [2]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="Can you play Taylor Swift's most popular song?")
for event in app.stream({"messages": [input_message]}, config=config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Can you play Taylor Swift's most popular song?
================================== Ai Message ==================================
Tool Calls:
  play_song_on_spotify (call_zojnECKI3X6GehDlKUEUfnOR)
 Call ID: call_zojnECKI3X6GehDlKUEUfnOR
  Args:
    song: Shake It Off by Taylor Swift
================================= Tool Message =================================
Name: play_song_on_spotify

Successfully played Shake It Off by Taylor Swift on Spotify
================================== Ai Message ==================================

I have successfully played "Shake It Off" by Taylor Swift on Spotify! Enjoy the music!


### Checking history

In [3]:
app.get_state(config).values["messages"]

[HumanMessage(content="Can you play Taylor Swift's most popular song?", additional_kwargs={}, response_metadata={}, id='d6a27bef-87f2-41ee-bee9-a680626f2cd1'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zojnECKI3X6GehDlKUEUfnOR', 'function': {'arguments': '{"song":"Shake It Off by Taylor Swift"}', 'name': 'play_song_on_spotify'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 80, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dfeeeedf-ed0a-45b4-9bd8-7b88fc32badd-0', tool_calls=[{'name': 'play_song_on_spotify', 'args': {'song': 'Shake It Off by Taylor Swift'}, 'id': 'call_zojnECKI

In [4]:
all_states = []
for state in app.get_state_history(config):
    print(state)
    all_states.append(state)
    print("--")

StateSnapshot(values={'messages': [HumanMessage(content="Can you play Taylor Swift's most popular song?", additional_kwargs={}, response_metadata={}, id='d6a27bef-87f2-41ee-bee9-a680626f2cd1'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zojnECKI3X6GehDlKUEUfnOR', 'function': {'arguments': '{"song":"Shake It Off by Taylor Swift"}', 'name': 'play_song_on_spotify'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 80, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dfeeeedf-ed0a-45b4-9bd8-7b88fc32badd-0', tool_calls=[{'name': 'play_song_on_spotify', 'args': {'song': 'Shake It Off by Tay

### Replay a state

In [ ]:
to_replay = all_states[2]
to_replay.value

{'messages': [HumanMessage(content="Can you play Taylor Swift's most popular song?", additional_kwargs={}, response_metadata={}, id='d6a27bef-87f2-41ee-bee9-a680626f2cd1'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zojnECKI3X6GehDlKUEUfnOR', 'function': {'arguments': '{"song":"Shake It Off by Taylor Swift"}', 'name': 'play_song_on_spotify'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 80, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dfeeeedf-ed0a-45b4-9bd8-7b88fc32badd-0', tool_calls=[{'name': 'play_song_on_spotify', 'args': {'song': 'Shake It Off by Taylor Swift'}, 'id': 

In [ ]:
to_replay.next

('action',)

In [8]:
for event in app.stream(None, to_replay.config):
    for v in event.values():
        print(v)

{'messages': [ToolMessage(content='Successfully played Shake It Off by Taylor Swift on Spotify', name='play_song_on_spotify', id='774fbf7f-3b1b-46f9-b8f5-dc6d7104be23', tool_call_id='call_zojnECKI3X6GehDlKUEUfnOR')]}
{'messages': [AIMessage(content='I\'ve successfully played "Shake It Off" by Taylor Swift on Spotify. Enjoy the music!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 123, 'total_tokens': 143, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-b1a93aa5-4a84-43bd-ada6-5dce41602f23-0', usage_metadata={'input_tokens': 123, 'output_tokens': 20, 'total_tokens': 143, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_tok

## Branch off a past state

In [9]:
last_message = to_replay.values["messages"][-1]
last_message.tool_calls[0]["name"] = "play_song_on_spotify"

branch_config = app.update_state(to_replay.config, {"messages": [last_message]})
for event in app.stream(None, branch_config):
    for v in event.values():
        print(v)

{'messages': [ToolMessage(content='Successfully played Shake It Off by Taylor Swift on Spotify', name='play_song_on_spotify', id='af71610d-10d3-4e79-a24b-a8868485c798', tool_call_id='call_zojnECKI3X6GehDlKUEUfnOR')]}
{'messages': [AIMessage(content='I\'ve successfully played "Shake It Off" by Taylor Swift on Spotify. Enjoy the music!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 123, 'total_tokens': 143, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-5b4f5837-e968-439e-adbe-8ab470384f94-0', usage_metadata={'input_tokens': 123, 'output_tokens': 20, 'total_tokens': 143, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_tok

Update state to not call tool:

In [10]:
from langchain_core.messages import AIMessage

last_message = to_replay.values["messages"][-1]
new_message = AIMessage(content="It's quire hours so I cant play any music right now!", id=last_message.id)
branch_config = app.update_state(to_replay.config, {"messages": [new_message]})

branch_state = app.get_state(branch_config)
branch_state.values

{'messages': [HumanMessage(content="Can you play Taylor Swift's most popular song?", additional_kwargs={}, response_metadata={}, id='d6a27bef-87f2-41ee-bee9-a680626f2cd1'),
  AIMessage(content="It's quire hours so I cant play any music right now!", additional_kwargs={}, response_metadata={}, id='run-dfeeeedf-ed0a-45b4-9bd8-7b88fc32badd-0')]}

In [11]:
branch_state.next

()